4) For the real instances, WELL-01.csv and WELL-02.csv, compute for each variable (P-PDG, P-TPT, T-TPT) of each class (0 and 2) the Binary Pattern (BP). For this, for each instance, you will split the multivariate time series into two parts: the data labeled as class 0 and the data labeled as class 2. After, compute for each variable of each class the BP with n = 5 (each histogram will have 16 bins). Show the normalized histogram of each variable of each class of each instance (for each instance will have 3 histograms for each class (a total of 12 histograms). Use the subplot function to become the visualization easier. Analyze the obtained histograms and check if it is possible to distinguish the classes by the histograms and if the behavior of the classes is consistent in the two instances. Comment on your observations.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# Definição das colunas de interesse
columns = ['Timestamp', 'P-PDG', 'P-TPT', 'T-TPT', 'class']

well_1 = pd.read_csv('WELL-01.csv', delimiter=',', usecols=columns)
well_2 = pd.read_csv('WELL-02.csv', delimiter=',', usecols=columns)

# Separação dos dados de acordo com índices fornecidos
split_idx_w1 = 3674
split_idx_w2 = 3594

w1c0 = well_1.iloc[:split_idx_w1, :].copy()
w1c2 = well_1.iloc[split_idx_w1:, :].reset_index(drop=True)

w2c0 = data02.iloc[:split_idx_w2, :].copy()
w2c2 = data02.iloc[split_idx_w2:, :].reset_index(drop=True)

# Exibir informações sobre os subconjuntos criados
print(f"w1c0: {w1c0.shape}, w1c2: {w1c2.shape}")
print(f"w2c0: {w2c0.shape}, w2c2: {w2c2.shape}")

















# Função para calcular o padrão binário de uma série
def bin_pattern(block, series):
    n = int(block)
    ts = pd.Series(series)
    gap = n // 2  # Melhor arredondamento
    pattern_list = np.zeros(len(ts) - (2 * gap))
    
    for i in range(gap, len(ts) - gap):
        for j in [-2, -1, 1, 2]:
            if ts.iloc[i + j] >= ts.iloc[i]:
                if j > 0:
                    pattern_list[i - gap] += 2 ** (j + 1)
                else:
                    pattern_list[i - gap] += 2 ** (j + 2)
    
    return pattern_list

